In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
#from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
#from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn import tree

# Obróbka danych


## wczytanie

In [25]:
dane = pd.read_csv('laptopPrice.csv')
dane = dane.iloc[:, 0:18]
dane.dropna()

,brand,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,os_bit,graphic_card_gb,weight,warranty,Touchscreen,msoffice,Price,rating,Number of Ratings
0,ASUS,Intel,Core i3,10th,4 GB,DDR4,0 GB,1024 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,34649,2 stars,3
1,Lenovo,Intel,Core i3,10th,4 GB,DDR4,0 GB,1024 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,38999,3 stars,65
2,Lenovo,Intel,Core i3,10th,4 GB,DDR4,0 GB,1024 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,39999,3 stars,8
3,ASUS,Intel,Core i5,10th,8 GB,DDR4,512 GB,0 GB,Windows,32-bit,2 GB,Casual,No warranty,No,No,69990,3 stars,0
4,ASUS,Intel,Celeron Dual,Not Available,4 GB,DDR4,0 GB,512 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,26990,3 stars,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,ASUS,AMD,Ryzen 9,Not Available,4 GB,DDR4,1024 GB,0 GB,Windows,64-bit,0 GB,Casual,1 year,No,No,135990,3 stars,0
819,ASUS,AMD,Ryzen 9,Not Available,4 GB,DDR4,1024 GB,0 GB,Windows,64-bit,0 GB,Casual,1 year,No,No,144990,3 stars,0
820,ASUS,AMD,Ryzen 9,Not Available,4 GB,DDR4,1024 GB,0 GB,Windows,64-bit,4 GB,Casual,1 year,No,No,149990,3 stars,0
821,ASUS,AMD,Ryzen 9,Not Available,4 GB,DDR4,1024 GB,0 GB,Windows,64-bit,4 GB,Casual,1 year,No,No,142990,3 stars,0


## Przekształcenie danych kategorycznych nominalnych

In [26]:
nom = ['brand', 'processor_brand', 'processor_name', 'ram_type', 'os', 'weight', 'Touchscreen', 'msoffice']

koder_nom = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
zakod_kateg = koder_nom.fit_transform(dane[nom])

zakod_dane = pd.DataFrame(zakod_kateg, columns=koder_nom.get_feature_names_out(nom))
newDane = pd.concat([dane, zakod_dane], axis=1).drop(columns=nom)


## Przekształcenie danych kategorycznych kolejnościowych

In [27]:
kol = ['processor_gnrtn', 'ram_gb', 'ssd', 'hdd', 'os_bit', 'graphic_card_gb', 'warranty', 'rating']

# szybka funkcja do poprawnego sortowania numerycznego
def sortkey(n):
    i = 0
    ret = ""
    while i < len(n) and n[i].isnumeric():
        ret += n[i]
        i+=1
    if ret == "" or ret == "0":
        return 0
    return math.log2(float(ret))


for i in kol:
    akt = sorted(newDane[i].unique(), key=sortkey)
    koder_akt = OrdinalEncoder(categories= [akt])
    newDane[i] = koder_akt.fit_transform(newDane[[i]])


# konwersja inr -> pln
for i in range(0, newDane.shape[0]):
    newDane.loc[i, 'Price'] *= 0.047   

print(newDane.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823 entries, 0 to 822
Data columns (total 48 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   processor_gnrtn              823 non-null    float64
 1   ram_gb                       823 non-null    float64
 2   ssd                          823 non-null    float64
 3   hdd                          823 non-null    float64
 4   os_bit                       823 non-null    float64
 5   graphic_card_gb              823 non-null    float64
 6   warranty                     823 non-null    float64
 7   Price                        823 non-null    float64
 8   rating                       823 non-null    float64
 9   Number of Ratings            823 non-null    int64  
 10  brand_APPLE                  823 non-null    float64
 11  brand_ASUS                   823 non-null    float64
 12  brand_Avita                  823 non-null    float64
 13  brand_DELL          

C:\Users\janma\AppData\Local\Temp\ipykernel_16192\764416333.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1628.503' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDane.loc[i, 'Price'] *= 0.047


## Old & useless


In [28]:
# przypisywanie wartości liczbowych od 1 do N
# poszczególnym wartościom kategorycznym (Intel -> 1, itp.)

# tablica wszystkich etykiet - l'importante
#etykiety = []

#for i in kolumny:
#    etykiety.append(pd.DataFrame(range(1, dane[i].unique().size + 1), index=np.sort(dane[i].unique())))


#rating = pd.DataFrame(range(1, 6), index=['1 star', '2 stars', '3 stars', '4 stars', '5 stars']) # -> retarded way

#newDane = pd.DataFrame.copy(dane)  #żeby zachować dane

#for j in range(0, len(etykiety)):
#    for i in range(0, newDane.shape[0]):
#        newDane.iloc[i,j] = etykiety[j].loc[newDane.iloc[i,j], 0] 


#for i in range(0, newDane.shape[0]):
#    newDane.iloc[i, 15] *= 0.047    # inr -> pln
#    newDane.iloc[i, 16] = newDane.iloc[i, 16][0]    # usunięcie "stars"


#print(etykiety)

#print(dane.head(10))
#print("======================================================================================================")
#print(newDane.head(10))

#sns.pairplot(newDane, kind="scatter", hue="brand", palette="bright")


# Uczenie modelu

## Podział zbioru na uczący i testowy

In [29]:
#print(newDane.info())
#print(newDane.head(1))
#plt.figure(figsize=(20,20), dpi=20)
#sns.pairplot(newDane, kind="scatter", hue="brand")

#for i in range(16, 17):
 #   kolumny.append(dane.columns[i])


X = newDane.drop(['Price'], axis=1)
y = newDane['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# coś się popsuło, to naprawiło
y_train = y_train.astype(int)
y_test = y_test.astype(int)


## Model & uczenie 

### Parametry

In [43]:
param_grid_rf = {
'n_estimators': [10, 50, 100, 500, 1000],
'max_depth': [3, 5, 7, 9, 11, 13]
}

param_grid_gb = {
'n_estimators': [10, 50, 100, 500, 750, 1000],
'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
'max_depth': [3, 5, 7, 9],
'min_samples_split': [5, 10, 20],
'subsample': [0.2, 0.5, 0.7, 1]
}

param_grid_hgb = {
'max_bins': [10, 20, 30, 40, 50, 100, 500],
'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
'max_depth': [3, 5, 7, 9, 11, 13, 15]
}

param_grid_ab = {
'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500],
'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0]
}

param_grid_kn = {
'n_neighbors': [5, 10, 15, 20, 50, 100],
}

# określanie parametrów zostało zakomentowane z uwagi na czas szukania parametrów

### Regresja  

#### Liniowa

In [31]:
lin = LinearRegression()
lin.fit(X_train, y_train)

predPrice = lin.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", lin.score(X_test, y_test))
cross_val_score(lin, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  657.3614929103578
Wynik:  0.7485690929345447


0.67089482928106

#### Random Forest

##### Określenie parametrów

In [32]:
#rfr = RandomForestRegressor()

#rfr2 = GridSearchCV(rfr, param_grid_rf, cv=3, n_jobs=-1)
#rfr2.fit(X_train, y_train)

#rfr2.best_params_

##### Model

In [33]:
rfr = RandomForestRegressor(n_estimators=50, max_depth=13)
rfr.fit(X_train, y_train)

predPrice = rfr.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", rfr.score(X_test, y_test))
cross_val_score(rfr, X_train, y_train, cv=3, n_jobs=-1).mean()



Błąd średniokwadratowy:  541.3147911224456
Wynik:  0.8284891756763466


0.7238780119196427

#### Gradient Boosting

##### Określanie parametrów

In [34]:
#gbr = GradientBoostingRegressor()

#gbr2 = GridSearchCV(gbr, param_grid_gb, cv=3, n_jobs=-1)
#gbr2.fit(X_train, y_train)

#gbr2.best_params_


##### Model

In [35]:
gbr = GradientBoostingRegressor(n_estimators=1000, max_depth=9, learning_rate=0.01, \
                                min_samples_split=25, min_samples_leaf=2, subsample=0.5, \
                                min_weight_fraction_leaf= 0.0)
gbr.fit(X_train, y_train)

predPrice = gbr.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", gbr.score(X_test, y_test))
cross_val_score(gbr, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  502.3822939074286
Wynik:  0.8500138716594807


0.759641175125826

#### Hist Gradient Boosting

In [36]:
#hgbr = HistGradientBoostingRegressor()

#hgbr2 = GridSearchCV(hgbr, param_grid_hgb, cv=3, n_jobs=-1)
#hgbr2.fit(X_train, y_train)

#hgbr2.best_params_

In [37]:
hgbr = HistGradientBoostingRegressor(learning_rate=0.1, max_depth=5, max_bins=50)
hgbr.fit(X_train, y_train)

predPrice = hgbr.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", hgbr.score(X_test, y_test))
cross_val_score(hgbr, X_train, y_train, cv=3, n_jobs=-1).mean()



Błąd średniokwadratowy:  611.5645191643731
Wynik:  0.7752832195150872


0.6911487525086529

#### Ada Boosting - weird

#####  Określenie parametrów


In [38]:
#ada = AdaBoostRegressor()

#ada2 = GridSearchCV(ada, param_grid_ab, cv=3, n_jobs=-1)
#ada2.fit(X_train, y_train)

#ada2.best_estimator_

##### Uczenie


In [39]:
ada = AdaBoostRegressor(n_estimators=1000, learning_rate=0.01)
ada.fit(X_train, y_train)

predPrice = ada.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", ada.score(X_test, y_test))
cross_val_score(ada, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  1022.8010015189238
Wynik:  0.6098912004837571


0.5425556742336789

### Klasyfikacja - generalnie raczej nie


#### Centroidy

In [40]:
mnp = NearestCentroid()
mnp.fit(X_train, y_train)

pred = mnp.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
cross_val_score(mnp, X_train, y_train, cv=3, n_jobs=-1).mean()



Błąd średniokwadratowy:  1022.8010015189238


c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


0.015206863152068631

#### KNN

In [49]:
knn = KNeighborsClassifier(n_neighbors=10)

knnn = GridSearchCV(knn, param_grid_kn, cv=3, n_jobs=-1)
knnn.fit(X_train, y_train)

knnn.best_params_

c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


{'n_neighbors': 10}

In [52]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

pred = knn.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, pred))
cross_val_score(knn, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  1417.0424242424242


c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


0.02127438771274388

#### Random Forest


##### Określenie parametrów

In [16]:
#rfc = RandomForestClassifier()

#rfc2 = GridSearchCV(rfc, param_grid_rf, cv=3)
#rfc2.fit(X_train, y_train)

#rfc2.best_params_

In [19]:
rfc = RandomForestClassifier(max_depth=7, n_estimators=50)
rfc.fit(X_train, y_train)

predPrice = rfc.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
cross_val_score(rfc, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  691.8545454545455


c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


0.08054517780545177

#### Gradient Boosting - jeszcze średniej

##### Params

In [18]:
#gbc = GradientBoostingClassifier()
#gbc2 = GridSearchCV(gbc, param_grid_gb, cv=3, n_jobs=-1)
#gbc2.fit(X_train, y_train)
#gbc2.best_params_

In [21]:
gbc = GradientBoostingClassifier(n_estimators=50, max_depth=3, learning_rate=0.1)
gbc.fit(X_train, y_train)

predPrice = gbc.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
cross_val_score(gbc, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  1541.430303030303


c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


0.05624740556247406

#### Ada Boosting

##### Params

In [20]:
#adc = AdaBoostClassifier()

#adc2 = GridSearchCV(adc, param_grid_ab, cv=3, n_jobs=-1)
#adc2.fit(X_train, y_train)

#adc2.best_estimator_

In [53]:
adc = AdaBoostClassifier(n_estimators=80, learning_rate=0.01)
adc.fit(X_train, y_train)

predPrice = adc.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
cross_val_score(adc, X_train, y_train, cv=3, n_jobs=-1).mean()

c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Błąd średniokwadratowy:  922.5454545454545


c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


0.0684032101840321

## Regresja z Wektorem Wspomagającym (SVR)

In [22]:
clf = svm.SVR(kernel='rbf', C=5000,  gamma='auto')
clf.fit(X_train, y_train)

predPrice = clf.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", clf.score(X_test, y_test))
cross_val_score(clf, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  850.1885487270106
Wynik:  0.5703197481973153


0.5470052599240683

## Ridge

In [23]:
clf = Ridge(alpha = 0.01)
clf.fit(X_train, y_train)

predPrice = clf.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", clf.score(X_test, y_test))
cross_val_score(clf, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  657.5663222406878
Wynik:  0.7486206183373838


0.6709119768715904

## Lasso

In [24]:
clf = Lasso(alpha=0.2, selection='random')
clf.fit(X_train, y_train)

predPrice = clf.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", clf.score(X_test, y_test))
cross_val_score(clf, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  657.7208865221669
Wynik:  0.7487364657175168


c:\Users\janma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.443e+08, tolerance: 2.968e+05
  model = cd_fast.enet_coordinate_descent(


0.6714885820665343

## ElasticNet

In [25]:
clf = ElasticNet(alpha=0.2, l1_ratio=1)
clf.fit(X_train, y_train)

predPrice = clf.predict(X_test)

print("Błąd średniokwadratowy: ", metrics.mean_absolute_error(y_test, predPrice))
print("Wynik: ", clf.score(X_test, y_test))
cross_val_score(clf, X_train, y_train, cv=3, n_jobs=-1).mean()

Błąd średniokwadratowy:  657.6821045549036
Wynik:  0.7487296164926212


0.6714814289063448